In [253]:
from bs4 import BeautifulSoup, NavigableString, Tag
import urllib
import requests
import re
import nltk

import pandas as pd

In [3]:
url = 'http://www.chakoteya.net/DS9/404.htm'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')

In [306]:
episode_title = soup.b.string
episode_title = episode_title.replace('\r\n', ' ')

In [307]:
episode_title

'Past Prologue'

In [46]:
array_of_strings = []
for string in soup.stripped_strings:
    array_of_strings.append(string.replace('\r\n', ' '))

In [47]:
array_of_strings[10:30]

["BASHIR: Would you care for some of this Tarkalean tea? It's very good.",
 "GARAK: What a thoughtful young man. How nice that we've met.",
 'BASHIR: You know, some people say that you remained on DS Nine as the eyes and ears of your fellow Cardassians.',
 "GARAK: You don't say? Doctor, you're not intimating that I'm considered some sort of spy, are you?",
 "BASHIR: I wouldn't know, sir.",
 "GARAK: Ah. An open mind. The essence of intellect. As you may also know, I have a clothing shop nearby, so if you should require any apparel, or merely wish, as I do, for a bit of enjoyable company now and then, I'm at your disposal, Doctor.",
 "BASHIR: You're very kind, Mister Garak.",
 "GARAK: Oh, it's just Garak. Plain, simple Garak. Now, good day to you, Doctor. I'm so glad to have made such an interesting new friend today.",
 '[Ops]',
 "BASHIR: You won't never believe who just sat down next to me at the Replimat",
 "O'BRIEN: Major, upper pylon three'll be shut down for maintenance for forty-ei

In [49]:
array_of_strings[0:10]

['The Deep Space Nine Transcripts - Past Prologue',
 'Past Prologue',
 'Stardate: Unknown',
 'Original Airdate: 11 Jan, 1993',
 '[Replimat]',
 "GARAK: It's Doctor Bashir, isn't it? Of course it is. May I introduce myself?",
 'BASHIR: Er, yes, yes, of course.',
 'GARAK: My name is Garak. Cardassian by birth, obviously. The only one of us left on this station, as a matter of fact, so I do appreciate making new friends whenever I can. You are new to this station, I believe.',
 "BASHIR: I am, yes. Though, though I understand you've been here quite a while.",
 'GARAK: Ah, you know of me then.']

In [300]:
#pattern = re.compile(r'([A-Z]+):')
#pattern = re.compile(r'(\b[A-Z]+)\b:')
#pattern = re.compile(r'(\b[A-Z]+)(\b:|\s\[)')
#pattern = re.compile(r'(\b[A-Z]+|[A-Z]+\'[A-Z]+)(\b:|\s\[)')
pattern = re.compile(r'(\b[A-Z]+|([A-Z]+.[A-Z]+))(\:|\s\[.+\]\:)')

In [228]:
for string in array_of_strings[0:10]:
    found = re.search(pattern, string)
    try:
        print('group0:', found.group(0), '\ngroup1:', found.group(1), '\n')
    except:
        print('No match found.\n')

No match found.

No match found.

No match found.

No match found.

No match found.

group0: GARAK: 
group1: GARAK 

group0: BASHIR: 
group1: BASHIR 

group0: GARAK: 
group1: GARAK 

group0: BASHIR: 
group1: BASHIR 

group0: GARAK: 
group1: GARAK 



In [133]:
small_df = array_of_strings[:50]

In [138]:
# for i, string in enumerate(small_df):
#     found = re.search(pattern, string)
#     try:
#         print(found.group(1), string)
#     except:
#         print('No speaker for {}'.format(string))
#         small_df.pop(i)

In [122]:
type(re.search(pattern, small_df[0]))

NoneType

In [230]:
# new_df = []
# for i, string in enumerate(small_df):
#     found = re.search(pattern, string)
#     if found is not None:
#         new_df.append(string)
#     else:
#         print('not appended:', string)

In [222]:
# clean_df = []
# char_dict = {}

# for string in array_of_strings:
#     found = re.search(pattern, string)
#     if found is not None:
#         clean_df.append(string)
        
#         try:
#             char_dict[found.group(1)] += 1
#         except:
#             char_dict[found.group(1)] = 1
        
#     else:
#         print('not appended:', string)


In [220]:
# clean_df = []
# char_dict = {}

# for string in array_of_strings:
#     found = re.search(pattern, string)
#     if found is not None:
#         stripped_string = string.replace(found.group(0), '').strip()
#         clean_df.append(stripped_string)
#         key = found.group(1)
#         char_dict.setdefault(key, []).append(stripped_string)
#         #dictionary.setdefault("list", []).append("list_item")
        
#     else:
#         print('not appended:', string)


In [219]:
sent_tokenizer = nltk.tokenize.sent_tokenize

In [301]:
# now with tokenizer sent_tokenizer
clean_df = []
char_dict = {}

for string in array_of_strings:
    found = re.search(pattern, string)
    if found is not None:
        stripped_string = string.replace(found.group(0), '').strip()
        stripped_string_tokenized = sent_tokenizer(stripped_string)
        
        key = found.group(1)
        
        for each in stripped_string_tokenized:
                clean_df.append(each)
                char_dict.setdefault(key, []).append(each)


Experimenting with converting dictionary to DataFrame:

In [273]:
pd.set_option('display.max_colwidth', -1) 

In [275]:
df = pd.DataFrame(char_dict['BASHIR'], columns=['text'])
df['character'] = 'Bashir'
df.head()

,text,character
0,"Er, yes, yes, of course.",Bashir
1,"I am, yes.",Bashir
2,"Though, though I understand you've been here quite a while.",Bashir
3,Would you care for some of this Tarkalean tea?,Bashir
4,It's very good.,Bashir


In [293]:
for key in char_dict:
    print(key)

GARAK
BASHIR
O'BRIEN
SISKO
DAX
KIRA
TAHNA
DANAR
ROLLMAN
ODO
DEPUTY
LURSA
B'ETOR


In [313]:
df = pd.DataFrame(columns=['character', 'ep_title', 'text'])

for key in char_dict:
    temp_df = pd.DataFrame(char_dict[key], columns=['text'])
    temp_df['character'] = key
    temp_df['ep_title'] = episode_title
    df = df.append(temp_df)

In [314]:
df

,character,ep_title,text
0,GARAK,Past Prologue,"It's Doctor Bashir, isn't it?"
1,GARAK,Past Prologue,Of course it is.
2,GARAK,Past Prologue,May I introduce myself?
3,GARAK,Past Prologue,My name is Garak.
4,GARAK,Past Prologue,"Cardassian by birth, obviously."
5,GARAK,Past Prologue,"The only one of us left on this station, as a matter of fact, so I do appreciate making new friends whenever I can."
6,GARAK,Past Prologue,"You are new to this station, I believe."
7,GARAK,Past Prologue,"Ah, you know of me then."
8,GARAK,Past Prologue,What a thoughtful young man.
9,GARAK,Past Prologue,How nice that we've met.


In [297]:
df.to_csv('past_prologue_df.csv', index=False)

In [298]:
new_df = pd.read_csv('./past_prologue_df.csv')

In [299]:
new_df

,text,character
0,"It's Doctor Bashir, isn't it?",GARAK
1,Of course it is.,GARAK
2,May I introduce myself?,GARAK
3,My name is Garak.,GARAK
4,"Cardassian by birth, obviously.",GARAK
5,"The only one of us left on this station, as a matter of fact, so I do appreciate making new friends whenever I can.",GARAK
6,"You are new to this station, I believe.",GARAK
7,"Ah, you know of me then.",GARAK
8,What a thoughtful young man.,GARAK
9,How nice that we've met.,GARAK
